# The best coffee on my way to work in Sydney, Australia

##  Introduction/Business Problem

Even though I am working from home now due to the COVID-19 pandemic, I know one day, things will go back to normal (well... semi normal?) and I will get back and work in the Sydney office again! As an addict to caffeine, I am looking forward to my morning coffee made by a professional barista. However, things are changing quickly including cafe and coffee shops. They might have changed ownership, barista, or even blending in a different way. I still remember once I had a bad experience spending like AUD5 for a cup of crap from a coffee shop that used to be my favourite!!! To avoid the same from happening again, I am going to take this opportunity to find out the best coffee shops between my train station and the office. I will run this query again on the day before I am back to the office to make sure it won't ruin my day!

## Overview of how data will be collected and examined

First I am going to find out the coordinates of the Townhall station and my work place. I will take the average of the coordinates as the mid-point, and use Foursquare to search for all the coffee shop/cafe within the radius. Then I will trim it down to the dataset with the name of the streets that I might be passing (I don't mind to go a bit further for a cup of perfect coffee). I believe in that area number of coffee shops will be less than 50 so it satisfy the searh LIMIT in Foursquare. Then, I will run queries to get the ratings of these coffee shops/cafes (venue) and only select the top to further proceed.

I have two particular coffee shops/cafes I enjoy very much with. Next I will make some Premium API Calls to find out tips and users who gave the tip. I will use the ratings done by these users as a criteria to trim down my final decision.

Well with a Personal Tier I have 99,500 Regular API Calls per day and 500 Premium API Calls per day. It should be enough to achieve this little project.

### Step 1, collect data from Foursqure in the area interested

In [1]:
%matplotlib inline

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [145]:
# Let's define the location coordinate of Townhall and Office.
venue_TH = 'Townhall Station'
address_TH = '63 George St, Sydney, Australia'
venue_OF = 'Office'
address_OF = '383 Kent St, Sydney, Australia'
latitude_TH = -33.873251
longitude_TH = 151.206718
latitude_OF = -33.869814
longitude_OF = 151.204269

print(venue_TH)
print('address:',address_TH)
print(latitude_TH, longitude_TH)
print('\n')
print(venue_OF)
print('address:',address_OF)
print(latitude_OF, longitude_OF)
print('\n')
print('This is the middle point:')
latitude_MID = (latitude_TH + latitude_OF)/2
longitude_MID = (longitude_TH + longitude_OF)/2
print(latitude_MID, longitude_MID)


Townhall Station
address: 63 George St, Sydney, Australia
-33.873251 151.206718


Office
address: 383 Kent St, Sydney, Australia
-33.869814 151.204269


This is the middle point:
-33.8715325 151.2054935


In [146]:
# Let's calculate the distance bewteen the two points: Townhall and Office. This middle point will be used for Foursquare searching and also for Folium map.

from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373

lat1 = radians(latitude_TH)
lon1 = radians(longitude_TH)
lat2 = radians(latitude_OF)
lon2 = radians(longitude_OF)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c *1000

print("Result:", distance,'m')

Result: 444.1901395782622 m


In [147]:
# let's get ready to retrieve data via API from Foursquare
CLIENT_ID = 'BESBPEDHNTKUY32TFLTWUP3HLLAG45FNSH4DB1PQI2KGPJP2' #  Foursquare ID
CLIENT_SECRET = 'MMB0U1FGJMTCDWEHNOI1YSWMKCGT1E0RBRH3MBBFKNEXLO2M' #  Foursquare Secret
VERSION = '20180604'
LIMIT = 50 #50 is the maximum limit anyway

# search radius is half of the distance between Townhall station and the office
search_query = 'coffee'
radius_search = distance / 2

In [148]:
# Let's define the search query.
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    latitude_MID,
    longitude_MID,
    VERSION,
    search_query,
    radius_search,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BESBPEDHNTKUY32TFLTWUP3HLLAG45FNSH4DB1PQI2KGPJP2&client_secret=MMB0U1FGJMTCDWEHNOI1YSWMKCGT1E0RBRH3MBBFKNEXLO2M&ll=-33.8715325,151.2054935&v=20180604&query=coffee&radius=222.0950697891311&limit=50'

In [149]:
results_all_coffee = requests.get(url).json()

In [150]:
# Get relevant part of JSON and transform it into a *pandas* dataframe
# assign relevant part of JSON to venues
venues_all_coffee = results_all_coffee['response']['venues']

# tranform venues into a dataframe
df_raw = json_normalize(venues_all_coffee)

In [151]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_raw.columns if col.startswith('location.')] + ['id']
df_filtered = df_raw.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]
df_filtered.head(50)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Foodoo Coffee Lounge,Coffee Shop,MYER 5th Floor (Pitt Street ),AU,Sydney,Australia,George Street,232,[MYER 5th Floor (Pitt Street ) (George Street)...,"[{'label': 'display', 'lat': -33.8703641546462...",-33.870364,151.207573,NaN,2000,NSW,51d7c52a498edf6da91d6007
1,Kent St Specialty Coffee,Café,414 Kent St.,AU,Sydney,Australia,Market St.,63,"[414 Kent St. (Market St.), Sydney NSW 2000, A...","[{'label': 'display', 'lat': -33.8718974149045...",-33.871897,151.204972,NaN,2000,NSW,4d8aae4d26a36ea8f39cfaad
2,Old Vienna Coffee House,Coffee Shop,455 George St.,AU,Sydney,Australia,NaN,114,"[455 George St., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8713127062593...",-33.871313,151.206705,NaN,2000,NSW,4b18474af964a5201cd023e3
3,Campos Coffee,Café,NaN,AU,NaN,Australia,NaN,221,[Australia],"[{'label': 'display', 'lat': -33.8731851049854...",-33.873185,151.206832,NaN,NaN,NaN,51440250e4b0ab7c89300305
4,Veneziano Caffee Speciality Coffee,Café,NaN,AU,Sydney,Australia,Kent Street,52,"[Kent Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8716605522493...",-33.871661,151.204942,NaN,2000,NSW,4f690c36e4b07a700b357b30
5,Gloria Jean's Coffees,Coffee Shop,The Galleries Victoria\rCnr Pitt & Park Streets,AU,Sydney,Australia,cnr Pitt St. & Park St.,216,[The Galleries Victoria\rCnr Pitt & Park Stree...,"[{'label': 'display', 'lat': -33.870428, 'lng'...",-33.870428,151.207425,NaN,2000,NSW,4b5aa658f964a5206bcf28e3
6,Mission Coffee,Café,NaN,AU,NaN,Australia,NaN,160,[Australia],"[{'label': 'display', 'lat': -33.8702140747043...",-33.870214,151.206195,NaN,NaN,NaN,55cbd122498eebb48c50f8e6
7,Market St Coffee,Coffee Shop,NaN,AU,Sydney,Australia,NaN,170,"[Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.870742, 'lng'...",-33.870742,151.207068,NaN,2000,NSW,59090e7ea6fe4d71fc743edc
8,Georgie Boy's Coffee Co.,Café,The Galleries,AU,Sydney,Australia,500 George St.,225,"[The Galleries (500 George St.), Sydney NSW 20...","[{'label': 'display', 'lat': -33.872654, 'lng'...",-33.872654,151.207530,NaN,2000,NSW,5c69ff52c9a517002c0b76f3
9,Vella Nero,Café,259 Clarence St.,AU,Sydney,Australia,btwn Market St. & Druitt St.,30,[259 Clarence St. (btwn Market St. & Druitt St...,"[{'label': 'display', 'lat': -33.8717699813282...",-33.871770,151.205665,NaN,2000,NSW,4b09f05df964a520242023e3


### Step 2, trim down number of coffee shops/cafes to those I may visit

In [152]:
# let's drop the Coffee schools
#df_filtered.drop(df_filtered.index[[5,13,27]], inplace = True)
#df_filtered = df_filtered.reset_index(drop=True)

In [153]:
# let's have a look on what we've got so far on a map.

coffee_map = folium.Map(location=[latitude_MID, longitude_MID], zoom_start=16) # generate map centred around the mid point

# add red circle markers to represent the Train Station and the Office
folium.features.CircleMarker(
    [latitude_TH, longitude_TH],
    radius=10,
    color='red',
    popup='Townhall Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

folium.features.CircleMarker(
    [latitude_OF, longitude_OF],
    radius=10,
    color='red',
    popup='Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

# add big circle represent the the search area in Foursquare

folium.Marker([latitude_MID, longitude_MID]).add_to(coffee_map)
folium.Circle([latitude_MID, longitude_MID],
                    radius=radius_search
                   ).add_to(coffee_map)

# add the coffee shop and Cafe as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(coffee_map)

# display map
coffee_map


In [64]:
# I would like to avoid going to the eastern side of George Street (the 5 dots with the largest longtitude)
# also I don't want to go Sussex street and anywhere further West (the two dots with the least longtitude)
# and also not anywhere South of Townhall (the 1 red dot, latitude < latitude_MID)

In [154]:
# sort data frame as per latitude
df_filtered = df_filtered.sort_values(by = 'lat')
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Adore Coffee,Coffee Shop,NaN,AU,NaN,Australia,NaN,270,[Australia],"[{'label': 'display', 'lat': -33.8739581932831...",-33.873958,151.205284,NaN,NaN,NaN,51ce302a498e48b06732c561
1,Campos Coffee,Café,NaN,AU,NaN,Australia,NaN,221,[Australia],"[{'label': 'display', 'lat': -33.8731851049854...",-33.873185,151.206832,NaN,NaN,NaN,51440250e4b0ab7c89300305
2,Speed Bar coffee Shop,Coffee Shop,29 Park Street,AU,Sydney,Australia,NaN,234,"[29 Park Street, Sydney NSW 2010, Australia]","[{'label': 'display', 'lat': -33.8731453913815...",-33.873145,151.207125,NaN,2010,NSW,4d70758095acb1f746ab06b9
3,Jet Bar Caffe,Café,"Shop G55, Queen Victoria Building",AU,Sydney,Australia,455 George St.,197,"[Shop G55, Queen Victoria Building (455 George...","[{'label': 'display', 'lat': -33.8729270769873...",-33.872927,151.206808,NaN,2000,NSW,4c0a5b37ed2595210a1d1356
4,Georgie Boy's Coffee Co.,Café,The Galleries,AU,Sydney,Australia,500 George St.,225,"[The Galleries (500 George St.), Sydney NSW 20...","[{'label': 'display', 'lat': -33.872654, 'lng'...",-33.872654,151.207530,NaN,2000,NSW,5c69ff52c9a517002c0b76f3


In [155]:
df_filtered.drop(df_filtered.index[[0]], inplace = True)


In [156]:
# sort data frame as per latitude
df_filtered = df_filtered.sort_values(by = 'lng')
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.head(30)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Adore Estate Coffee,Coffee Shop,Clarence,AU,Sydney,Australia,NaN,298,"[Clarence, Sydney NSW, Australia]","[{'label': 'display', 'lat': -33.8689828088546...",-33.868983,151.204498,NaN,NaN,NSW,4ee7cddb9a5288af4817de01
1,Veneziano Caffee Speciality Coffee,Café,NaN,AU,Sydney,Australia,Kent Street,52,"[Kent Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8716605522493...",-33.871661,151.204942,NaN,2000,NSW,4f690c36e4b07a700b357b30
2,Kent St Specialty Coffee,Café,414 Kent St.,AU,Sydney,Australia,Market St.,63,"[414 Kent St. (Market St.), Sydney NSW 2000, A...","[{'label': 'display', 'lat': -33.8718974149045...",-33.871897,151.204972,NaN,2000,NSW,4d8aae4d26a36ea8f39cfaad
3,Pure Grind Café,Café,225 Clarence Street,AU,Sydney,Australia,NaN,112,"[225 Clarence Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8705260317789...",-33.870526,151.205550,Sydney City Center,2000,NSW,4b6f3173f964a5205be32ce3
4,Klink Handmade Espresso,Coffee Shop,281 Clarence St,AU,Sydney,Australia,nr Druitt St,89,"[281 Clarence St (nr Druitt St), Sydney NSW 20...","[{'label': 'display', 'lat': -33.872331, 'lng'...",-33.872331,151.205593,NaN,2000,NSW,4b8b0961f964a520858f32e3
5,Vella Nero,Café,259 Clarence St.,AU,Sydney,Australia,btwn Market St. & Druitt St.,30,[259 Clarence St. (btwn Market St. & Druitt St...,"[{'label': 'display', 'lat': -33.8717699813282...",-33.871770,151.205665,NaN,2000,NSW,4b09f05df964a520242023e3
6,Mission Coffee,Café,NaN,AU,NaN,Australia,NaN,160,[Australia],"[{'label': 'display', 'lat': -33.8702140747043...",-33.870214,151.206195,NaN,NaN,NaN,55cbd122498eebb48c50f8e6
7,Old Vienna Coffee House,Coffee Shop,455 George St.,AU,Sydney,Australia,NaN,114,"[455 George St., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8713127062593...",-33.871313,151.206705,NaN,2000,NSW,4b18474af964a5201cd023e3
8,Starbucks,Coffee Shop,"T69, Lower Ground Floor, Queen Victoria Building",AU,Sydney,Australia,at 455 George St,138,"[T69, Lower Ground Floor, Queen Victoria Build...","[{'label': 'display', 'lat': -33.8722137670364...",-33.872214,151.206754,NaN,2000,NSW,4b0b7029f964a520793123e3
9,Bacio Café,Café,"Ground Level, Queen Victoria Building",AU,Sydney,Australia,455 George St.,119,"[Ground Level, Queen Victoria Building (455 Ge...","[{'label': 'display', 'lat': -33.8715367931891...",-33.871537,151.206783,NaN,2000,NSW,4b5145d5f964a520734927e3


In [157]:
df_filtered.drop(df_filtered.index[[12,13,14,15,16]], inplace = True)
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.head(30)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Adore Estate Coffee,Coffee Shop,Clarence,AU,Sydney,Australia,NaN,298,"[Clarence, Sydney NSW, Australia]","[{'label': 'display', 'lat': -33.8689828088546...",-33.868983,151.204498,NaN,NaN,NSW,4ee7cddb9a5288af4817de01
1,Veneziano Caffee Speciality Coffee,Café,NaN,AU,Sydney,Australia,Kent Street,52,"[Kent Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8716605522493...",-33.871661,151.204942,NaN,2000,NSW,4f690c36e4b07a700b357b30
2,Kent St Specialty Coffee,Café,414 Kent St.,AU,Sydney,Australia,Market St.,63,"[414 Kent St. (Market St.), Sydney NSW 2000, A...","[{'label': 'display', 'lat': -33.8718974149045...",-33.871897,151.204972,NaN,2000,NSW,4d8aae4d26a36ea8f39cfaad
3,Pure Grind Café,Café,225 Clarence Street,AU,Sydney,Australia,NaN,112,"[225 Clarence Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8705260317789...",-33.870526,151.205550,Sydney City Center,2000,NSW,4b6f3173f964a5205be32ce3
4,Klink Handmade Espresso,Coffee Shop,281 Clarence St,AU,Sydney,Australia,nr Druitt St,89,"[281 Clarence St (nr Druitt St), Sydney NSW 20...","[{'label': 'display', 'lat': -33.872331, 'lng'...",-33.872331,151.205593,NaN,2000,NSW,4b8b0961f964a520858f32e3
5,Vella Nero,Café,259 Clarence St.,AU,Sydney,Australia,btwn Market St. & Druitt St.,30,[259 Clarence St. (btwn Market St. & Druitt St...,"[{'label': 'display', 'lat': -33.8717699813282...",-33.871770,151.205665,NaN,2000,NSW,4b09f05df964a520242023e3
6,Mission Coffee,Café,NaN,AU,NaN,Australia,NaN,160,[Australia],"[{'label': 'display', 'lat': -33.8702140747043...",-33.870214,151.206195,NaN,NaN,NaN,55cbd122498eebb48c50f8e6
7,Old Vienna Coffee House,Coffee Shop,455 George St.,AU,Sydney,Australia,NaN,114,"[455 George St., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8713127062593...",-33.871313,151.206705,NaN,2000,NSW,4b18474af964a5201cd023e3
8,Starbucks,Coffee Shop,"T69, Lower Ground Floor, Queen Victoria Building",AU,Sydney,Australia,at 455 George St,138,"[T69, Lower Ground Floor, Queen Victoria Build...","[{'label': 'display', 'lat': -33.8722137670364...",-33.872214,151.206754,NaN,2000,NSW,4b0b7029f964a520793123e3
9,Bacio Café,Café,"Ground Level, Queen Victoria Building",AU,Sydney,Australia,455 George St.,119,"[Ground Level, Queen Victoria Building (455 Ge...","[{'label': 'display', 'lat': -33.8715367931891...",-33.871537,151.206783,NaN,2000,NSW,4b5145d5f964a520734927e3


In [158]:
# let's have a look on what we've got so far on a map.

coffee_map = folium.Map(location=[latitude_MID, longitude_MID], zoom_start=16) # generate map centred around the mid point

# add red circle markers to represent the Train Station and the Office
folium.features.CircleMarker(
    [latitude_TH, longitude_TH],
    radius=10,
    color='red',
    popup='Townhall Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

folium.features.CircleMarker(
    [latitude_OF, longitude_OF],
    radius=10,
    color='red',
    popup='Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

# add big circle represent the the search area in Foursquare

folium.Marker([latitude_MID, longitude_MID]).add_to(coffee_map)
folium.Circle([latitude_MID, longitude_MID],
                    radius=radius_search
                   ).add_to(coffee_map)

# add the coffee shop and Cafe as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(coffee_map)

# display map
coffee_map


In [159]:
# now I am happy with the coffe shops I have. Let's rename the dataframe.
df_coffee = df_filtered

### Step 3, get the ratings of these coffee shops, explore the venues

In [160]:
#let's first add a column to the dataframe to get the ratings
df_coffee = df_coffee.assign(rating='')

In [161]:
#let's have a quick of the rating, and at the same time add the rating to the dataframe.

for idx, name, venue_id in zip(df_coffee.index, df_coffee.name, df_coffee.id):
    
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
        venue_id,
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION)
    url
    result = requests.get(url).json()
    print(name)
    try:
        print(result['response']['venue']['rating']),
        df_coffee.set_value(idx,'rating',result['response']['venue']['rating'])
    except:
        print('no rating.'),
        df_coffee.set_value(idx,'rating','no rating')

Adore Estate Coffee
no rating.
Veneziano Caffee Speciality Coffee
no rating.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Kent St Specialty Coffee
5.9


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Pure Grind Café
no rating.
Klink Handmade Espresso
8.1
Vella Nero
7.3
Mission Coffee
no rating.
Old Vienna Coffee House
4.9
Starbucks
7.2
Bacio Café
6.2
Jet Bar Caffe
6.0
Campos Coffee
no rating.


In [162]:
# well, it is a bit disppointing that only 8 out of 16 coffee shops have been rated.
df_coffee.head(16)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,rating
0,Adore Estate Coffee,Coffee Shop,Clarence,AU,Sydney,Australia,NaN,298,"[Clarence, Sydney NSW, Australia]","[{'label': 'display', 'lat': -33.8689828088546...",-33.868983,151.204498,NaN,NaN,NSW,4ee7cddb9a5288af4817de01,no rating
1,Veneziano Caffee Speciality Coffee,Café,NaN,AU,Sydney,Australia,Kent Street,52,"[Kent Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8716605522493...",-33.871661,151.204942,NaN,2000,NSW,4f690c36e4b07a700b357b30,no rating
2,Kent St Specialty Coffee,Café,414 Kent St.,AU,Sydney,Australia,Market St.,63,"[414 Kent St. (Market St.), Sydney NSW 2000, A...","[{'label': 'display', 'lat': -33.8718974149045...",-33.871897,151.204972,NaN,2000,NSW,4d8aae4d26a36ea8f39cfaad,5.9
3,Pure Grind Café,Café,225 Clarence Street,AU,Sydney,Australia,NaN,112,"[225 Clarence Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8705260317789...",-33.870526,151.205550,Sydney City Center,2000,NSW,4b6f3173f964a5205be32ce3,no rating
4,Klink Handmade Espresso,Coffee Shop,281 Clarence St,AU,Sydney,Australia,nr Druitt St,89,"[281 Clarence St (nr Druitt St), Sydney NSW 20...","[{'label': 'display', 'lat': -33.872331, 'lng'...",-33.872331,151.205593,NaN,2000,NSW,4b8b0961f964a520858f32e3,8.1
5,Vella Nero,Café,259 Clarence St.,AU,Sydney,Australia,btwn Market St. & Druitt St.,30,[259 Clarence St. (btwn Market St. & Druitt St...,"[{'label': 'display', 'lat': -33.8717699813282...",-33.871770,151.205665,NaN,2000,NSW,4b09f05df964a520242023e3,7.3
6,Mission Coffee,Café,NaN,AU,NaN,Australia,NaN,160,[Australia],"[{'label': 'display', 'lat': -33.8702140747043...",-33.870214,151.206195,NaN,NaN,NaN,55cbd122498eebb48c50f8e6,no rating
7,Old Vienna Coffee House,Coffee Shop,455 George St.,AU,Sydney,Australia,NaN,114,"[455 George St., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8713127062593...",-33.871313,151.206705,NaN,2000,NSW,4b18474af964a5201cd023e3,4.9
8,Starbucks,Coffee Shop,"T69, Lower Ground Floor, Queen Victoria Building",AU,Sydney,Australia,at 455 George St,138,"[T69, Lower Ground Floor, Queen Victoria Build...","[{'label': 'display', 'lat': -33.8722137670364...",-33.872214,151.206754,NaN,2000,NSW,4b0b7029f964a520793123e3,7.2
9,Bacio Café,Café,"Ground Level, Queen Victoria Building",AU,Sydney,Australia,455 George St.,119,"[Ground Level, Queen Victoria Building (455 Ge...","[{'label': 'display', 'lat': -33.8715367931891...",-33.871537,151.206783,NaN,2000,NSW,4b5145d5f964a520734927e3,6.2


In [163]:
df_rated = df_coffee[df_coffee.rating!='no rating']

In [168]:
df_rated = df_rated.sort_values(by = 'rating', ascending=False)
df_rated = df_rated.reset_index(drop=True)
df_rated.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,rating
0,Klink Handmade Espresso,Coffee Shop,281 Clarence St,AU,Sydney,Australia,nr Druitt St,89,"[281 Clarence St (nr Druitt St), Sydney NSW 20...","[{'label': 'display', 'lat': -33.872331, 'lng'...",-33.872331,151.205593,NaN,2000,NSW,4b8b0961f964a520858f32e3,8.1
1,Vella Nero,Café,259 Clarence St.,AU,Sydney,Australia,btwn Market St. & Druitt St.,30,[259 Clarence St. (btwn Market St. & Druitt St...,"[{'label': 'display', 'lat': -33.8717699813282...",-33.871770,151.205665,NaN,2000,NSW,4b09f05df964a520242023e3,7.3
2,Starbucks,Coffee Shop,"T69, Lower Ground Floor, Queen Victoria Building",AU,Sydney,Australia,at 455 George St,138,"[T69, Lower Ground Floor, Queen Victoria Build...","[{'label': 'display', 'lat': -33.8722137670364...",-33.872214,151.206754,NaN,2000,NSW,4b0b7029f964a520793123e3,7.2
3,Bacio Café,Café,"Ground Level, Queen Victoria Building",AU,Sydney,Australia,455 George St.,119,"[Ground Level, Queen Victoria Building (455 Ge...","[{'label': 'display', 'lat': -33.8715367931891...",-33.871537,151.206783,NaN,2000,NSW,4b5145d5f964a520734927e3,6.2
4,Jet Bar Caffe,Café,"Shop G55, Queen Victoria Building",AU,Sydney,Australia,455 George St.,197,"[Shop G55, Queen Victoria Building (455 George...","[{'label': 'display', 'lat': -33.8729270769873...",-33.872927,151.206808,NaN,2000,NSW,4c0a5b37ed2595210a1d1356,6
5,Kent St Specialty Coffee,Café,414 Kent St.,AU,Sydney,Australia,Market St.,63,"[414 Kent St. (Market St.), Sydney NSW 2000, A...","[{'label': 'display', 'lat': -33.8718974149045...",-33.871897,151.204972,NaN,2000,NSW,4d8aae4d26a36ea8f39cfaad,5.9
6,Old Vienna Coffee House,Coffee Shop,455 George St.,AU,Sydney,Australia,NaN,114,"[455 George St., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8713127062593...",-33.871313,151.206705,NaN,2000,NSW,4b18474af964a5201cd023e3,4.9


In [165]:
# let's have a look on what we've got so far on a map.

coffee_map = folium.Map(location=[latitude_MID, longitude_MID], zoom_start=16) # generate map centred around the mid point

# add red circle markers to represent the Train Station and the Office
folium.features.CircleMarker(
    [latitude_TH, longitude_TH],
    radius=10,
    color='red',
    popup='Townhall Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

folium.features.CircleMarker(
    [latitude_OF, longitude_OF],
    radius=10,
    color='red',
    popup='Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

# add big circle represent the the search area in Foursquare

folium.Marker([latitude_MID, longitude_MID]).add_to(coffee_map)
folium.Circle([latitude_MID, longitude_MID],
                    radius=radius_search
                   ).add_to(coffee_map)

# add the coffee shop and Cafe as blue circle markers
for lat, lng, label, rating, rank in zip(df_rated.lat, df_rated.lng, df_rated.name, df_rated.rating, df_rated.index):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup='shop: {}, rating: {}, ranking: {}'.format(label,rating,rank+1),
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(coffee_map)

# display map
coffee_map